In [240]:
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver import ActionChains
import pandas as pd
import time
import numpy as np

In [728]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_class(clas):
    try:
        driver.find_element_by_class_name(clas)
    except NoSuchElementException:
        return False
    return True

In [763]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [254]:
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and (not y%100==0 or y%400 == 0) else 28,
        31,30,31,30,31,31,30,31,30,31][m-1])
    return date.replace(day=1,month=m, year=y)



In [860]:
def partidas (pos):
    temp_codigo=data['codigo'].iloc[pos]
    partida=driver.find_element_by_id("txtNandinaB")
    partida.send_keys(temp_codigo)
    
    time.sleep(3)
    #partidaelegida =  driver.find_element_by_class_name("ui-menu-item-wrapper")
    #partidaelegida.click()
    
    partidaelegida = driver.find_element_by_css_selector('li.ui-menu-item')
    partidaelegida.click()
    

In [831]:
def grilla_df():
    xpath=[]
    for tran in range(7, 1, -1):
        xpath='//*[@id="pagingTabDetalleExcel"]/li['+str(tran)+']/a'
        if check_exists_by_xpath(xpath):
            break

    ContinueExecuting = True
    while ContinueExecuting:
        #Codigo extra aqui dependiendo de la situacion
        add_to_tabla_matrix()
        time.sleep(5)
        if not check_exists_by_class("page-item.next.disabled"):
            paginado = driver.find_element_by_xpath(xpath)
            paginado.click() # get the data and write it to scrapy items
            time.sleep(5)             
        else:
            ContinueExecuting = False




In [876]:
 def iteracciones():
    print('entro interacciones')
    cont=0
    rango_data_it= (data['codigo'].count()/10)
    rango_data_it= int(np.ceil(rango_data_it))
    for j in range(rango_data_it):
        for i in range(10):
            partidas(cont)
            temp_codigo=data['codigo'].iloc[i]
            # print ("Nro "+ str(cont) +" -codigo:"+temp_codigo)
            cont=cont+1
            if((i+1) % 10 == 0):
                break
            if cont == len(data):
                break
        time.sleep(10)
        exportar()
        clickrestablecer= driver.find_element_by_id("btnRestablecer")
        clickrestablecer.click()#//*[@id="btnRestablecer"]
        time.sleep(10)
    print('finalizo interacciones')
    


In [816]:
def add_to_tabla_matrix():
    table = driver.find_element_by_id("gridDetalleExcel")
           # now get all the TR elements from the table
    all_rows = table.find_elements_by_tag_name("tr")
    global df
    temp=[]
    for row in all_rows:
        nro=''      # and iterate over them, getting the cells
        fecha=''
        partida=''
        exportador=''
        importador=''
        kgneto=''
        cantidad=''
        unid=''
        fob=''
        dua=''
        pais=''
        descripcion=''
        cells = row.find_elements_by_tag_name("td")
        cont=0
        for value in cells:
            if cont==0:
                nro= value.text
            if cont==1:
                fecha= value.text
            if cont==2:
                partida= value.text
            if cont==3:
                exportador= value.text
            if cont==4:
                importador= value.text
            if cont==5:
                kgneto= value.text
            if cont==6:
                cantidad= value.text
            if cont==7:
                unid= value.text
            if cont==8:
                fob= value.text
            if cont==9:
                dua= value.text
            if cont==10:
                pais= value.text
            if cont==11:
                descripcion= value.text
            temp.append(str(cont)+"†"+value.text )
            cont=cont+1
            if cont==12:
                break    
        if nro:
            df=df.append({'nro':nro,'fecha':fecha,'partida':partida,'exportador':exportador,'importador':importador,'kgneto':kgneto,'cantidad':cantidad,'unid':unid,'fob':fob,'dua':dua,'pais':pais,'descripcion':descripcion}, ignore_index=True)
        #de= de.append({'nro':1,'fecha':2,'partida':3},ignore_index=True)
   

    
            


In [874]:
def exportar():
    print('entro exportar')
    calendarioini=driver.find_element_by_xpath('//*[@id="cboDesde"]/div/span') 
    calendarioini.click()
    
    time.sleep(1)
    if hoy.month<=2:
        previous=driver.find_element_by_xpath('/html/body/div[7]/div[2]/table/thead/tr[2]/th[1]')
        ActionChains(driver).click(previous).perform()
    time.sleep(1)
    python_botton = driver.find_elements_by_class_name("month")[mes]
    python_botton.click()
    time.sleep(1)
    calendariofin=driver.find_element_by_xpath('//*[@id="cboHasta"]/div/span') 
    calendariofin.click()
    time.sleep(1)
    python_botton = driver.find_elements_by_class_name("month")[hoy.month-1]
    python_botton.click()
    time.sleep(1)
    clickbuscar= driver.find_element_by_id("btnBuscar")
    clickbuscar.click()
    time.sleep(5)
            
    clickbuscar= driver.find_element_by_id("tabDetalleExcel")
    clickbuscar.click()#//*[@id="tabDetalleExcel"]

            
    time.sleep(5)
    clickexportar= driver.find_element_by_id("downloadFileVerRegistro2")
    clickexportar.click()#//*[@id="downloadFileVerRegistro2"]/img
    time.sleep(10)
    
    print('finalizo exportar')

    

In [852]:
#Variables
df = pd.DataFrame(columns=['nro','fecha','partida','exportador','importador','kgneto','cantidad','unid','fob','dua','pais','descripcion'])
data = pd.read_excel("Partidas.xlsx")
data = data[data['x']==0]
hoy =  date.today()
Fechabuscada= monthdelta(hoy, -2)
mes = Fechabuscada.month-1
año = Fechabuscada.year 

#Inicio
driver=webdriver.Chrome('./chromedriver.exe')
driver.get('https://www.veritradecorp.com/')
clicklogin= driver.find_element_by_id("li_login")
clicklogin.click()
time.sleep(5)

usuario=driver.find_element_by_id("txtCodUsuario")
usuario.send_keys("pmartinto@villaandina.com")
contraseña = driver.find_element_by_id("txtPassword")
contraseña.send_keys("VILLAANDINA1")

clicklogin2= driver.find_element_by_id("login_button")
clicklogin2.click()

time.sleep(5)
clickradioexp =driver.find_element_by_id("rdbExp")
clickradioexp.click()




In [877]:
#Exportacion
time.sleep(10)
iteracciones()
time.sleep(5)



entro interacciones
entro exportar
finalizo exportar
entro exportar
finalizo exportar
finalizo interacciones


In [422]:
#Cierre de Sesión
ruedaconfig=driver.find_element_by_xpath('//*[@id="bs-example-navbar-collapse-1"]/ul[2]/li[2]/a/span') 
ruedaconfig.click()
time.sleep(3)
cerrarsesion=driver.find_element_by_xpath('//*[@id="bs-example-navbar-collapse-1"]/ul[2]/li[2]/ul/li[9]/a') 
cerrarsesion.click()




In [804]:
df = pd.DataFrame(columns=['nro','fecha','partida','exportador',
                           'importador','kgneto','cantidad','unid','fob','dua','pais','descripcion'])